In [45]:
import json
import re

def extract_solution_text(sample):
    # Match the block that starts after "### Solution JSON:"
    # and extract everything from the first '{' to the matching '}'
    match = re.search(r'###\s*Solution\s*JSON:\s*(\{.*)', sample['generated_text'], re.DOTALL)
    
    if not match:
        # Try fallback: look for any JSON-like object
        match = re.search(r'(\{[^{}]*"solution"\s*:\s*".*?"[^{}]*\})', sample['generated_text'], re.DOTALL)
    
    if not match:
        return None, "No JSON block found"

    json_str = match.group(1).strip()
    json_str = match.group(1).replace('\n', ' ').replace('\r', '') + '}'

    try:
        parsed = json.loads(json_str)
        return parsed.get("solution", "").strip().lower(), None
    except Exception as e:
        print(sample['generated_text'])
        return None, f"JSON parse error: {e}"

In [39]:
sample = """
Below are some Multiple Choice Questions. Write responses in Arabic language only that appropriately complete each request in a valid, parsable JSON format with two attributes, one will be \"reasoning\" which is your thought process, \n    the other is the \"solution\" that has only a letter (a, b, c or d) in English, which represents the option you chose for the solution based on the options provided in the question.\n\n### Question:\nمن هو الصحابي الذي كانت تستحي منه ملائكة السماء؟\n\n\n### Options:\na. علي بن أبي طالب\nb. عثمان بن عفان\nc. عمر بن الخطاب\nd. أبو بكر الصديق\n\n\n### Solution JSON:\n{\n    \"reasoning\": \" الصحابة الذين كانوا يُعتقد أنهم كانوا مع الصحابة المذكورين في القرآن، كان منهم عمر بن الخطاب وأبو بكر الصديق. عمر بن الخطاب كان يُعتقد أن الملائكة كانوا تستحي منه، بينما أبو بكر الصديق كان يُعتقد أن الملائكة كانت تستحي منه. \n   \n   فيما يلي الحل: c. عمر بن الخطاب\",\n   \n    \"solution\": \"c. عمر بن الخطاب\"
"""
match = re.search(r'###\s*Solution\s*JSON:\s*(\{.*)', sample, re.DOTALL)
print(match.group(1).replace('\n', ' ').replace('\r', ''))    
json_str = match.group(1).replace('\n', ' ').replace('\r', '') + '}'
print(json_str)
parsed = json.loads(json_str)


{     "reasoning": " الصحابة الذين كانوا يُعتقد أنهم كانوا مع الصحابة المذكورين في القرآن، كان منهم عمر بن الخطاب وأبو بكر الصديق. عمر بن الخطاب كان يُعتقد أن الملائكة كانوا تستحي منه، بينما أبو بكر الصديق كان يُعتقد أن الملائكة كانت تستحي منه.         فيما يلي الحل: c. عمر بن الخطاب",         "solution": "c. عمر بن الخطاب" 
{     "reasoning": " الصحابة الذين كانوا يُعتقد أنهم كانوا مع الصحابة المذكورين في القرآن، كان منهم عمر بن الخطاب وأبو بكر الصديق. عمر بن الخطاب كان يُعتقد أن الملائكة كانوا تستحي منه، بينما أبو بكر الصديق كان يُعتقد أن الملائكة كانت تستحي منه.         فيما يلي الحل: c. عمر بن الخطاب",         "solution": "c. عمر بن الخطاب" }


In [43]:
def evaluate_from_file(file_path):
    total = 0
    correct = 0
    with open(file_path, "r", encoding="utf-8") as f:
        
        for line in f:
            sample = json.loads(line)
            is_correct, pred = extract_solution_text(sample)
            total += 1
            if is_correct:
                correct += 1
            else:
                print(f"[✘] ID {sample.get('id')}: predicted = {pred}, expected = {sample.get('answer_key')}")
    
    print(f"\nAccuracy: {correct}/{total} = {correct / total:.2%}")

# Example usage
evaluate_from_file("generations.jsonl")  # replace with your actual file

[✘] ID 526: predicted = JSON parse error: Expecting ',' delimiter: line 1 column 160 (char 159), expected = b
[✘] ID 1278: predicted = JSON parse error: Unterminated string starting at: line 1 column 20 (char 19), expected = a
[✘] ID 1713: predicted = JSON parse error: Expecting property name enclosed in double quotes: line 1 column 381 (char 380), expected = c
[✘] ID 1934: predicted = JSON parse error: Expecting property name enclosed in double quotes: line 1 column 7 (char 6), expected = d
[✘] ID 2250: predicted = JSON parse error: Unterminated string starting at: line 1 column 20 (char 19), expected = a
[✘] ID 2252: predicted = JSON parse error: Unterminated string starting at: line 1 column 20 (char 19), expected = a
[✘] ID 2381: predicted = JSON parse error: Unterminated string starting at: line 1 column 20 (char 19), expected = a
[✘] ID 2386: predicted = JSON parse error: Unterminated string starting at: line 1 column 20 (char 19), expected = c
[✘] ID 2407: predicted = JSON parse